<a href="https://colab.research.google.com/github/rafavidal1709/projeto-aplicado-iii/blob/main/02%20-%20Acur%C3%A1cia%20dos%20modelos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Carregando o dataset

Baixe o arquivo "dataset_embedding.json" (ou utilize o gerado ao fim da etapa anterior) e carregue-o abaixo para que possamos testar a acurácia dos modelos.

_O arquivo deve levar até alguns minutos para fazer o upload para o notebook._

In [2]:
import json
from google.colab import files

# Carrega o dataset
def upload_dataset():
  uploaded = files.upload()
  return json.loads(uploaded[list(uploaded.keys())[0]].decode('utf-8'))

df = upload_dataset()
dataset_base = df['base']
dataset_real = df['real']

dataset_base['embedding'] = {int(k): v for k, v in dataset_base['embedding'].items()}
dataset_real['embedding'] = {int(k): v for k, v in dataset_real['embedding'].items()}

# Utilizaremos para analisar o resultado
def print_accuracy(dataset):
  for method in dataset['accuracy'].keys():
    print(f'{method}:')
    for model in dataset['accuracy'][method].keys():
      print(f'\t{model}: {dataset["accuracy"][method][model]["mean"]}')

Saving 02 - dataset_embeddings.json to 02 - dataset_embeddings.json


# Métrica de acurácia 1

Primeiro aplicamos a similaridade de cossenos ou o KNN para medir a acurácia de encontarmos os textos mais similares no sistema de recomendação. Os textos da mesma categoria têm um teor semelhante, sendo assim deveriam ser colocados mais próximos no hiperplano do embedding.

Como métrica vamos encontrar os oito textos mais similares que pertencem a mesma categoria e somar seus índices. Na situação ideal deveria ser: 0, 1, 2, 3, 4, 5, 6, 7; o que soma 28. Por isso a diferença entre o valor obtido com o ideal define o quão impreciso é o modelo, sendo assim quanto mais longe de zero pior é o modelo utilizado.

In [3]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def accuracy_test_cosine(dataset, embedding, top_n=8):
    # Inicializando a matriz de precisão com o shape (len(dataset['text']), top_n)
    accu_m = np.empty((len(dataset['text']), top_n))

    # Valor mínimo de acumulação
    min_accu = (top_n-1)*top_n/2

    embeddings = []
    for i in range(len(dataset['text'])):
        # Extraindo os embeddings correspondentes para o texto i
        embeddings.append(dataset['embedding'][i][embedding])

    for i in range(len(dataset['text'])):
        # Calculando similaridade cosseno entre o embedding atual e todos os outros embeddings
        similarities = cosine_similarity([embeddings[i]], embeddings)
        ranked_indices = np.argsort(similarities[0])[::-1]  # Ordenar por similaridade

        # Preenchendo a matriz de precisão
        n = 0
        for j in range(len(dataset['text'])):
            if dataset['category'][i] == dataset['category'][ranked_indices[j]]:
                accu_m[i][n] = j  # Salvando o índice da correspondência
                n += 1
            if n == top_n:
                break

    # Somando as linhas da matriz de precisão
    accu = np.sum(accu_m, axis=1).reshape(-1, 1)

    # Subtraindo o valor mínimo
    accu = accu - min_accu

    # Calculando precisão por categoria
    categories = list(set(dataset['category']))

    accu_mean = np.mean(accu)
    accu_per_cat = {}

    for category in categories:
        indices = np.where(np.array(dataset['category']) == category)
        accu_per_cat[category] = np.mean(accu[indices])  # Calcula a média para a categoria

    return {embedding: {"mean": accu_mean, "mean_per_cat": accu_per_cat, "matrix": accu_m.tolist()}}

In [4]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

def accuracy_test_knn(dataset, embedding, top_n=8):
    # Inicializando a matriz de precisão com o shape (len(dataset['text']), top_n)
    accu_m = np.empty((len(dataset['text']), top_n))

    # Valor mínimo de acumulação
    min_accu = (top_n-1)*top_n/2

    embeddings = []
    for i in range(len(dataset['text'])):
        # Extraindo os embeddings correspondentes para o texto i
        embeddings.append(dataset['embedding'][i][embedding])

    # Convertendo a lista de embeddings para um array do numpy
    embeddings = np.array(embeddings)

    # Usando NearestNeighbors para calcular os k vizinhos mais próximos (k=top_n)
    knn = NearestNeighbors(n_neighbors=len(dataset['text']), metric='euclidean')
    knn.fit(embeddings)

    for i in range(len(dataset['text'])):
        # Encontrando os k vizinhos mais próximos para o embedding atual
        distances, ranked_indices = knn.kneighbors([embeddings[i]])

        # Preenchendo a matriz de precisão
        n = 0
        for j in range(len(dataset['text'])):  # Considerando apenas os top_n vizinhos
            if dataset['category'][i] == dataset['category'][ranked_indices[0][j]]:
                accu_m[i][n] = j  # Salvando o índice da correspondência
                n += 1
            if n == top_n:
                break

    # Somando as linhas da matriz de precisão
    accu = np.sum(accu_m, axis=1).reshape(-1, 1)

    # Subtraindo o valor mínimo
    accu = accu - min_accu

    # Calculando precisão por categoria
    categories = list(set(dataset['category']))

    accu_mean = np.mean(accu)
    accu_per_cat = {}

    for category in categories:
        indices = np.where(np.array(dataset['category']) == category)
        accu_per_cat[category] = np.mean(accu[indices])  # Calcula a média para a categoria

    return {embedding: {"mean": accu_mean, "mean_per_cat": accu_per_cat, "matrix": accu_m.tolist()}}

# Avaliação dos modelos 1



Para cada modelo temos a média (mean), a média por categoria e por último a matriz resultante de todas as iterações do processo. Cada modelo também cria dois tipos de embedding o 'pooler output' e o 'output mean', ambos são abordagens comuns utilizadas nessa situação.

In [10]:
dataset_base['accuracy']['cosine_similarity']={}
for emb in dataset_base['embedding'][0].keys():
  for key, value in accuracy_test_cosine(dataset_base, emb).items():
    dataset_base['accuracy']['cosine_similarity'][key] = value

display(dataset_base['accuracy']['cosine_similarity'])

{'longformer_global_attention': {'mean': 107.6,
  'mean_per_cat': {'trabalhadores_sem_epi': 86.8,
   'contaminacao_nascente_agrotoxico': 122.65,
   'agrotoxico_proximo_residencias': 115.7,
   'queimadas': 93.7,
   'pisoteamento_nascente': 119.15},
  'matrix': [[0.0, 4.0, 9.0, 16.0, 17.0, 19.0, 21.0, 27.0],
   [0.0, 3.0, 5.0, 12.0, 19.0, 21.0, 25.0, 28.0],
   [0.0, 8.0, 12.0, 20.0, 21.0, 26.0, 31.0, 33.0],
   [0.0, 2.0, 20.0, 29.0, 32.0, 36.0, 42.0, 45.0],
   [0.0, 10.0, 13.0, 17.0, 25.0, 33.0, 41.0, 45.0],
   [0.0, 6.0, 8.0, 11.0, 13.0, 20.0, 27.0, 35.0],
   [0.0, 6.0, 8.0, 19.0, 23.0, 28.0, 31.0, 32.0],
   [0.0, 7.0, 13.0, 17.0, 28.0, 29.0, 32.0, 35.0],
   [0.0, 4.0, 7.0, 16.0, 17.0, 21.0, 27.0, 32.0],
   [0.0, 2.0, 15.0, 27.0, 29.0, 30.0, 31.0, 45.0],
   [0.0, 8.0, 9.0, 12.0, 20.0, 22.0, 35.0, 40.0],
   [0.0, 12.0, 14.0, 28.0, 30.0, 31.0, 38.0, 39.0],
   [0.0, 2.0, 12.0, 13.0, 17.0, 22.0, 41.0, 46.0],
   [0.0, 6.0, 14.0, 15.0, 20.0, 26.0, 31.0, 32.0],
   [0.0, 6.0, 7.0, 8.0, 16.0, 19

A segunda métrica, muito mais leve de ser calculada é o KNN. O resultado abaixo segue o mesmo padrão do anterior com média, média por categoria e matrix resultante.

In [11]:
dataset_base['accuracy']['knn_euclidian']={}
for emb in dataset_base['embedding'][0].keys():
  for key, value in accuracy_test_knn(dataset_base, emb).items():
    dataset_base['accuracy']['knn_euclidian'][key] = value

display(dataset_base['accuracy']['knn_euclidian'])

{'longformer_global_attention': {'mean': 107.6,
  'mean_per_cat': {'trabalhadores_sem_epi': 86.8,
   'contaminacao_nascente_agrotoxico': 122.65,
   'agrotoxico_proximo_residencias': 115.7,
   'queimadas': 93.7,
   'pisoteamento_nascente': 119.15},
  'matrix': [[0.0, 4.0, 9.0, 16.0, 17.0, 19.0, 21.0, 27.0],
   [0.0, 3.0, 5.0, 12.0, 19.0, 21.0, 25.0, 28.0],
   [0.0, 8.0, 12.0, 20.0, 21.0, 26.0, 31.0, 33.0],
   [0.0, 2.0, 20.0, 29.0, 32.0, 36.0, 42.0, 45.0],
   [0.0, 10.0, 13.0, 17.0, 25.0, 33.0, 41.0, 45.0],
   [0.0, 6.0, 8.0, 11.0, 13.0, 20.0, 27.0, 35.0],
   [0.0, 6.0, 8.0, 19.0, 23.0, 28.0, 31.0, 32.0],
   [0.0, 7.0, 13.0, 17.0, 28.0, 29.0, 32.0, 35.0],
   [0.0, 4.0, 7.0, 16.0, 17.0, 21.0, 27.0, 32.0],
   [0.0, 2.0, 15.0, 27.0, 29.0, 30.0, 31.0, 45.0],
   [0.0, 8.0, 9.0, 12.0, 20.0, 22.0, 35.0, 40.0],
   [0.0, 12.0, 14.0, 28.0, 30.0, 31.0, 38.0, 39.0],
   [0.0, 2.0, 12.0, 13.0, 17.0, 22.0, 41.0, 46.0],
   [0.0, 6.0, 14.0, 15.0, 20.0, 26.0, 31.0, 32.0],
   [0.0, 6.0, 7.0, 8.0, 16.0, 19

Na comparação entre os dois modelo multilinguas - BERT e Longformer -, o BERT apresentou melhor desempenho com 49.62 contra 87.64, representando um desempenho quase duas vezes melhor. Já o BERTimbau, que já é treinado em português, conseguiu uma média de 5.24, superando todos os demais modelos.

Se compararmos o KNN com o cossine similarity podemos concluir que os resultados são praticmante identicos tendo apenas pequenas variações insignificantes, o que nos faz escolher pelo KNN pela significante maior rapidez em realizar os cálculos com este método.

In [22]:
print_accuracy(dataset_base)

cosine_similarity:
	longformer_global_attention: 107.6
	longformer_output_mean: 87.64
	bert_pooler_output: 104.98
	bert_output_mean: 49.62
	bertimbau_pooler_output: 53.18
	bertimbau_output_mean: 5.24
knn_euclidian:
	longformer_global_attention: 107.6
	longformer_output_mean: 87.71
	bert_pooler_output: 105.07
	bert_output_mean: 50.2
	bertimbau_pooler_output: 53.61
	bertimbau_output_mean: 5.22


# Métrica de acurácia 2

Após o sucesso do primeiro teste quisemos testar o modelo em situações realistas. Os exemplos utilizados acima são denúncias ambientais que foram limpos para compor a base de dados de referência. Para não haver tendenciosidade em misturar fatores como nome de indivíduos e locais com o teor do que está sendo reportado, essas informações foram limpas do texto.

Já abaixo vamos continuar utilizando a mesma base de dados como referência, porém para medir a similaridade com outro conjunto de dados rotulado nas mesmas cinco categorias e com nomes e locais.

In [12]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def accuracy2_test_cosine(dataset, dataset2, embedding, top_n=8):
    # Inicializando a matriz de precisão com o shape (len(dataset['text']), top_n)
    accu_m = np.empty((len(dataset2['text']), top_n))

    # Valor mínimo de acumulação
    min_accu = (top_n-1)*top_n/2

    embeddings = []
    for i in range(len(dataset['text'])):
        # Extraindo os embeddings correspondentes para o texto i
        embeddings.append(dataset['embedding'][i][embedding])

    embeddings2 = []
    for i in range(len(dataset2['text'])):
        # Extraindo os embeddings correspondentes para o texto i
        embeddings2.append(dataset2['embedding'][i][embedding])

    for i in range(len(dataset2['text'])):
        # Calculando similaridade cosseno entre o embedding atual e todos os outros embeddings
        similarities = cosine_similarity([embeddings2[i]], embeddings)
        ranked_indices = np.argsort(similarities[0])[::-1]  # Ordenar por similaridade

        # Preenchendo a matriz de precisão
        n = 0
        for j in range(len(dataset['text'])):
            if dataset2['category'][i] == dataset['category'][ranked_indices[j]]:
                accu_m[i][n] = j  # Salvando o índice da correspondência
                n += 1
            if n == top_n:
                break

    # Somando as linhas da matriz de precisão
    accu = np.sum(accu_m, axis=1).reshape(-1, 1)

    # Subtraindo o valor mínimo
    accu = accu - min_accu

    # Calculando precisão por categoria
    categories = list(set(dataset2['category']))

    accu_mean = np.mean(accu)
    accu_per_cat = {}

    for category in categories:
        indices = np.where(np.array(dataset2['category']) == category)
        accu_per_cat[category] = np.mean(accu[indices])  # Calcula a média para a categoria

    return {embedding: {"mean": accu_mean, "mean_per_cat": accu_per_cat, "matrix": accu_m.tolist()}}

In [13]:
from sklearn.neighbors import NearestNeighbors
import numpy as np

def accuracy2_test_knn(dataset, dataset2, embedding, top_n=8):
    # Inicializando a matriz de precisão com o shape (len(dataset['text']), top_n)
    accu_m = np.empty((len(dataset2['text']), top_n))

    # Valor mínimo de acumulação
    min_accu = (top_n-1)*top_n/2

    embeddings = []
    for i in range(len(dataset['text'])):
        # Extraindo os embeddings correspondentes para o texto i
        embeddings.append(dataset['embedding'][i][embedding])

    # Convertendo a lista de embeddings para um array do numpy
    embeddings = np.array(embeddings)

    embeddings2 = []
    for i in range(len(dataset2['text'])):
        # Extraindo os embeddings correspondentes para o texto i
        embeddings2.append(dataset2['embedding'][i][embedding])

    # Convertendo a lista de embeddings para um array do numpy
    embeddings2 = np.array(embeddings2)

    # Usando NearestNeighbors para calcular os k vizinhos mais próximos (k=top_n)
    knn = NearestNeighbors(n_neighbors=len(dataset['text']), metric='euclidean')
    knn.fit(embeddings)

    for i in range(len(dataset2['text'])):
        # Encontrando os k vizinhos mais próximos para o embedding atual
        distances, ranked_indices = knn.kneighbors([embeddings2[i]])

        # Preenchendo a matriz de precisão
        n = 0
        for j in range(len(dataset['text'])):  # Considerando apenas os top_n vizinhos
            if dataset['category'][i] == dataset['category'][ranked_indices[0][j]]:
                accu_m[i][n] = j  # Salvando o índice da correspondência
                n += 1
            if n == top_n:
                break

    # Somando as linhas da matriz de precisão
    accu = np.sum(accu_m, axis=1).reshape(-1, 1)

    # Subtraindo o valor mínimo
    accu = accu - min_accu

    # Calculando precisão por categoria
    categories = list(set(dataset2['category']))

    accu_mean = np.mean(accu)
    accu_per_cat = {}

    for category in categories:
        indices = np.where(np.array(dataset2['category']) == category)
        accu_per_cat[category] = np.mean(accu[indices])  # Calcula a média para a categoria

    return {embedding: {"mean": accu_mean, "mean_per_cat": accu_per_cat, "matrix": accu_m.tolist()}}

# Avaliação dos modelos 2

In [14]:
dataset_real['accuracy']['cosine_similarity']={}
for emb in dataset_real['embedding'][0].keys():
  for key, value in accuracy2_test_cosine(dataset_base, dataset_real, emb).items():
    dataset_real['accuracy']['cosine_similarity'][key] = value

display(dataset_real['accuracy']['cosine_similarity'])

{'longformer_global_attention': {'mean': 140.4,
  'mean_per_cat': {'trabalhadores_sem_epi': 161.4,
   'contaminacao_nascente_agrotoxico': 153.1,
   'agrotoxico_proximo_residencias': 143.7,
   'queimadas': 110.6,
   'pisoteamento_nascente': 133.2},
  'matrix': [[6.0, 7.0, 19.0, 25.0, 34.0, 38.0, 40.0, 42.0],
   [2.0, 5.0, 9.0, 23.0, 26.0, 27.0, 37.0, 40.0],
   [1.0, 16.0, 17.0, 18.0, 23.0, 25.0, 29.0, 34.0],
   [10.0, 13.0, 15.0, 16.0, 31.0, 34.0, 35.0, 39.0],
   [0.0, 7.0, 11.0, 24.0, 28.0, 32.0, 36.0, 44.0],
   [3.0, 4.0, 13.0, 16.0, 21.0, 22.0, 23.0, 32.0],
   [0.0, 6.0, 7.0, 14.0, 20.0, 24.0, 26.0, 27.0],
   [4.0, 6.0, 7.0, 25.0, 29.0, 30.0, 39.0, 40.0],
   [7.0, 14.0, 16.0, 21.0, 22.0, 35.0, 36.0, 39.0],
   [12.0, 16.0, 17.0, 18.0, 20.0, 24.0, 27.0, 37.0],
   [9.0, 10.0, 11.0, 23.0, 28.0, 39.0, 42.0, 43.0],
   [1.0, 3.0, 11.0, 18.0, 37.0, 39.0, 40.0, 44.0],
   [1.0, 6.0, 12.0, 18.0, 35.0, 38.0, 43.0, 45.0],
   [4.0, 9.0, 10.0, 14.0, 17.0, 19.0, 24.0, 28.0],
   [0.0, 2.0, 13.0, 24.0

In [15]:
dataset_real['accuracy']['knn_euclidian']={}
for emb in dataset_real['embedding'][0].keys():
  for key, value in accuracy2_test_knn(dataset_base, dataset_real, emb).items():
    dataset_real['accuracy']['knn_euclidian'][key] = value

display(dataset_real['accuracy']['knn_euclidian'])

{'longformer_global_attention': {'mean': 144.44,
  'mean_per_cat': {'trabalhadores_sem_epi': 136.8,
   'contaminacao_nascente_agrotoxico': 142.8,
   'agrotoxico_proximo_residencias': 143.7,
   'queimadas': 157.1,
   'pisoteamento_nascente': 141.8},
  'matrix': [[6.0, 7.0, 19.0, 25.0, 34.0, 38.0, 40.0, 42.0],
   [2.0, 5.0, 9.0, 23.0, 26.0, 27.0, 37.0, 40.0],
   [1.0, 16.0, 17.0, 18.0, 23.0, 25.0, 29.0, 34.0],
   [10.0, 13.0, 15.0, 16.0, 31.0, 34.0, 35.0, 39.0],
   [0.0, 7.0, 11.0, 24.0, 28.0, 32.0, 36.0, 44.0],
   [3.0, 4.0, 13.0, 16.0, 21.0, 22.0, 23.0, 32.0],
   [0.0, 6.0, 7.0, 14.0, 20.0, 24.0, 26.0, 27.0],
   [4.0, 6.0, 7.0, 25.0, 29.0, 30.0, 39.0, 40.0],
   [7.0, 14.0, 16.0, 21.0, 22.0, 35.0, 36.0, 39.0],
   [12.0, 16.0, 17.0, 18.0, 20.0, 24.0, 27.0, 37.0],
   [4.0, 7.0, 8.0, 25.0, 27.0, 31.0, 35.0, 40.0],
   [0.0, 8.0, 13.0, 14.0, 29.0, 33.0, 41.0, 43.0],
   [0.0, 8.0, 15.0, 16.0, 23.0, 28.0, 41.0, 42.0],
   [0.0, 6.0, 13.0, 16.0, 27.0, 29.0, 33.0, 36.0],
   [4.0, 6.0, 8.0, 18.0, 

Ao adicionar essas informações vemos uma queda vertiginosa na acurácia, tendo o melhor resultado sendo 106.32 utiilizando ainda o BERTimbau com cosine similarity, porém o modelo multilinguas do BERT se equiparou ao modelo Longformer.

Vemos neste caso uma diferença significativa entre o uso o cosine_similarity e o KNN, sendo o melhor desempenho com o primeiro 106.32 e com o segundo 126.2.

In [23]:
print_accuracy(dataset_real)

cosine_similarity:
	longformer_global_attention: 140.4
	longformer_output_mean: 134.14
	bert_pooler_output: 145.76
	bert_output_mean: 150.46
	bertimbau_pooler_output: 129.44
	bertimbau_output_mean: 106.32
knn_euclidian:
	longformer_global_attention: 144.44
	longformer_output_mean: 137.52
	bert_pooler_output: 126.2
	bert_output_mean: 136.88
	bertimbau_pooler_output: 145.6
	bertimbau_output_mean: 148.86


# Conclusão

Testamos os métodos Longformer e BERT da arquitetura transformers e pudemos concluir que o BERT e o BERTimbau são melhores para lidar com a tarefa de embedding. Porém, pelos resultados que temos até o momento, seria preciso uma análise mais profunda da versão especializada em português e multilingua do BERT para definir qual seria a mais apropriada.

Mesmo para decidir entre o método de embedding pooler output e output mean, não podemos definir com exatidão o melhor. Porém o melhor desempenho registrado ao tratar dados mais realistas foi utilizando o pooler output do modelo multilinguas do BERT.

In [16]:
df = {'base': dataset_base, 'real': dataset_real}
json.dump(df, open('dataset_accuracy.json', 'w'))
files.download('dataset_accuracy.json')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>